# Regras de Associação — Dados de Jogadores da NBA
## Descobrindo Padrões de Desempenho


## 1. Objetivo de Negócio

O objetivo deste notebook é aplicar técnicas de **mineração de regras de associação** para identificar **padrões frequentes e relações** entre métricas de desempenho dos jogadores da NBA.

Ao descobrir combinações de atributos que ocorrem frequentemente em conjunto, esta análise pretende:
- Identificar perfis de desempenho comuns  
- Compreender quais as características estatísticas que tendem a ocorrer em conjunto  
- Apoiar decisões de scouting e definição de perfis de jogadores


In [1]:
import pandas as pd

df = pd.read_csv("nba_data.csv")
df.head()


,normalized_name,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,pts,...,FT.,PER,TS.,TRB.,AST.,TOV.,USG.,WS,VORP,BPM
0,Allen Iverson,26,183.58,74.61,Georgetown,USA,1996,1,1,31.1,...,0.814,24.0,0.518,5.2,23.0,10.0,35.9,11.8,6.1,6.1
1,Jerry Stackhouse,26,198.15,99.23,North Carolina,USA,1995,1,3,29.8,...,0.822,21.8,0.521,5.2,25.8,12.5,35.2,9.2,5.8,5.1
2,Shaquille O'Neal,29,216.31,142.56,Louisiana State,USA,1992,1,1,28.7,...,0.513,30.2,0.574,18.1,18.8,10.5,31.6,14.9,7.1,7.7
3,Kobe Bryant,22,201.60,95.20,NaN,USA,1996,1,13,28.5,...,0.853,24.5,0.552,8.1,23.0,11.1,31.8,11.3,4.7,4.8
4,Vince Carter,24,198.64,102.34,North Carolina,USA,1998,1,5,27.6,...,0.765,25.0,0.551,7.9,19.2,8.2,30.7,12.9,7.2,7.6


## 2. Preparação dos Dados

Algoritmos de regras de associação, como o **Apriori**, requerem **dados categóricos ou binários**.  
Portanto, as variáveis numéricas contínuas foram **discretizadas em níveis categóricos**.


In [2]:
features = [
    "pts",
    "reb",
    "ast",
    "PER",
    "TS.",
    "USG.",
    "WS",
    "VORP"
]

df_selected = df[features].copy()
df_selected = df_selected.fillna(df_selected.median())


In [3]:
for col in df_selected.columns:
    df_selected[col] = pd.qcut(
        df_selected[col],
        q=3,
        labels=[f"{col}_Low", f"{col}_Medium", f"{col}_High"]
    )


In [4]:
df_transactions = pd.get_dummies(df_selected)
df_transactions.head()


,pts_pts_Low,pts_pts_Medium,pts_pts_High,reb_reb_Low,reb_reb_Medium,reb_reb_High,ast_ast_Low,ast_ast_Medium,ast_ast_High,PER_PER_Low,...,TS._TS._High,USG._USG._Low,USG._USG._Medium,USG._USG._High,WS_WS_Low,WS_WS_Medium,WS_WS_High,VORP_VORP_Low,VORP_VORP_Medium,VORP_VORP_High
0,False,False,True,False,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,True
1,False,False,True,False,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,True
2,False,False,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True
3,False,False,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True
4,False,False,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True


## 3. Aplicação do Algoritmo Apriori


In [5]:
# Install mlxtend if needed
!pip install mlxtend


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 9.9 MB/s  0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   --------- ------------------------------ 2.9/12.4 MB 13.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.8/12.4 MB 13.3 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.4 MB 13.4 MB/s eta 0:00:01
   ------------------------------------ --- 11.3/12.4 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 12.6 MB/s  0:00:00

  Attempting uninstall: numpy

    Found existing installation: numpy 2.3.4

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
    Uninstalling numpy-2.3.4:
   ---------------------------------------- 0/

  You can safely remove it manually.
  You can safely remove it manually.


In [6]:
from mlxtend.frequent_patterns import apriori, association_rules


In [7]:
frequent_itemsets = apriori(
    df_transactions,
    min_support=0.1,
    use_colnames=True
)

frequent_itemsets.sort_values("support", ascending=False).head()


,support,itemsets
6,0.356956,(ast_ast_Low)
18,0.345680,(WS_WS_Low)
12,0.338511,(TS._TS._Low)
3,0.337465,(reb_reb_Low)
10,0.337316,(PER_PER_Medium)


## 4. Geração de Regras de Associação


In [8]:
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.6
)

rules.sort_values("lift", ascending=False).head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1256,"(WS_WS_High, USG._USG._High)","(ast_ast_High, pts_pts_High, VORP_VORP_High, P...",0.156822,0.148757,0.110522,0.704762,4.737684,1.0,0.087194,2.883244,0.935658,0.566616,0.653168,0.723867
1233,"(ast_ast_High, pts_pts_High, VORP_VORP_High, P...","(WS_WS_High, USG._USG._High)",0.148757,0.156822,0.110522,0.742972,4.737684,1.0,0.087194,3.280490,0.926793,0.566616,0.695168,0.723867
1255,"(VORP_VORP_High, USG._USG._High)","(WS_WS_High, pts_pts_High, ast_ast_High, PER_P...",0.168471,0.138601,0.110522,0.656028,4.733231,1.0,0.087172,2.504275,0.948528,0.562310,0.600683,0.726721
1237,"(WS_WS_High, pts_pts_High, ast_ast_High, PER_P...","(VORP_VORP_High, USG._USG._High)",0.138601,0.168471,0.110522,0.797414,4.733231,1.0,0.087172,4.104567,0.915635,0.562310,0.756369,0.726721
1239,"(WS_WS_High, VORP_VORP_High, USG._USG._High)","(pts_pts_High, ast_ast_High, PER_PER_High)",0.147263,0.158763,0.110522,0.750507,4.727206,1.0,0.087142,3.371786,0.924621,0.565317,0.703421,0.723325


## 5. Otimização dos Parâmetros do Apriori

Foram testados diferentes **limiares de support e confidence** para equilibrar o **número de regras geradas** e a sua **interpretabilidade**.


In [9]:
frequent_itemsets_opt = apriori(
    df_transactions,
    min_support=0.05,
    use_colnames=True
)

rules_opt = association_rules(
    frequent_itemsets_opt,
    metric="lift",
    min_threshold=1.2
)

rules_opt.sort_values("confidence", ascending=False).head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
10408,"(TS._TS._High, WS_WS_High, USG._USG._High, PER...",(VORP_VORP_High),0.063699,0.332835,0.063550,0.997655,2.997443,1.0,0.042349,284.545665,0.711718,0.190850,0.996486,0.594295
10975,"(TS._TS._High, WS_WS_High, USG._USG._High, pts...",(VORP_VORP_High),0.063475,0.332835,0.063326,0.997647,2.997418,1.0,0.042199,283.544918,0.711545,0.190177,0.996473,0.593955
10222,"(TS._TS._High, WS_WS_High, reb_reb_High, PER_P...",(VORP_VORP_High),0.059219,0.332835,0.059070,0.997478,2.996910,1.0,0.039359,264.530730,0.708266,0.177394,0.996220,0.587476
7973,"(TS._TS._High, ast_ast_High, PER_PER_High, WS_...",(VORP_VORP_High),0.086252,0.332835,0.086028,0.997403,2.996683,1.0,0.057320,256.858338,0.729192,0.258296,0.996107,0.627936
10597,"(TS._TS._High, WS_WS_High, pts_pts_High, reb_r...",(VORP_VORP_High),0.056904,0.332835,0.056755,0.997375,2.996602,1.0,0.037815,254.189680,0.706491,0.170442,0.996066,0.583947


## 6. Análise dos Resultados

As regras descobertas revelam **relações significativas** entre métricas de desempenho dos jogadores.

Exemplos de padrões identificados:
- Jogadores com **alta taxa de utilização (usage rate)** frequentemente apresentam também **altos pontos por jogo**  
- **PER elevado** ocorre frequentemente em conjunto com **Win Shares elevado**  
- Jogadores com **VORP elevado** tendem a ter também **TS% elevado**

Estas regras destacam **perfis estatísticos típicos de jogadores impactantes** na NBA.


## 7. Documentação dos Resultados Intermediários e Finais

- O **número de conjuntos de itens frequentes** descobertos varia significativamente com o limiar de support  
- **Valores de support mais baixos** aumentam a quantidade de regras, mas reduzem a interpretabilidade  
- O **Lift** revelou-se uma métrica útil para identificar associações não triviais  

A configuração final dos parâmetros atingiu um **equilíbrio entre relevância das regras e clareza**.


## 8. Conclusão e Lições Aprendidas

Este notebook demonstrou a aplicação da **mineração de regras de associação** em dados de jogadores da NBA, destacando como técnicas de **aprendizagem não supervisionada** podem revelar padrões ocultos na análise desportiva.

### Principais Lições Aprendidas:
- Os **dados numéricos desportivos** devem ser cuidadosamente discretizados antes da aplicação do Apriori  
- O **ajuste de parâmetros** é crucial para evitar a geração de regras excessivas ou triviais  
- As **regras de associação** complementam classificação e clustering, fornecendo **padrões interpretáveis**

De forma geral, a mineração de regras de associação revelou-se uma **ferramenta exploratória valiosa** para compreender os perfis de desempenho dos jogadores.
